In [ ]:
!python -V

Python 3.12.7


#### mlflow ui --backend-store-uri sqlite:///mlflow.db

In [1]:
import pandas as pd

In [2]:
import pickle

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [ ]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/home/habeeb/Mlops-proj/02-experimental_tracking/mlruns/1', creation_time=1747104414113, experiment_id='1', last_update_time=1747104414113, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [ ]:
mlflow --version

In [6]:
import importlib.util
import sys
import os

# Define full path to the module
module_path = "../01-intro/NYC_trip_duration_pred.py"  # adjust as needed

# Load the module dynamically
spec = importlib.util.spec_from_file_location("NYCTaxiModule", module_path)
module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(module)


nyc = module.NYCTaxiDurationPredictor()


In [7]:
df_train = nyc.download_data(year=2021, month=1, taxi='green')
df_train

Downloaded 76518 records


,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2021-01-01 00:15:56,2021-01-01 00:19:52,N,1.0,43,151,1.0,1.01,5.50,0.50,0.5,0.00,0.00,None,0.3,6.80,2.0,1.0,0.00
1,2,2021-01-01 00:25:59,2021-01-01 00:34:44,N,1.0,166,239,1.0,2.53,10.00,0.50,0.5,2.81,0.00,None,0.3,16.86,1.0,1.0,2.75
2,2,2021-01-01 00:45:57,2021-01-01 00:51:55,N,1.0,41,42,1.0,1.12,6.00,0.50,0.5,1.00,0.00,None,0.3,8.30,1.0,1.0,0.00
3,2,2020-12-31 23:57:51,2021-01-01 00:04:56,N,1.0,168,75,1.0,1.99,8.00,0.50,0.5,0.00,0.00,None,0.3,9.30,2.0,1.0,0.00
4,2,2021-01-01 00:16:36,2021-01-01 00:16:40,N,2.0,265,265,3.0,0.00,-52.00,0.00,-0.5,0.00,0.00,None,-0.3,-52.80,3.0,1.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76513,2,2021-01-31 21:38:00,2021-01-31 22:16:00,None,NaN,81,90,NaN,17.63,56.23,2.75,0.0,0.00,6.12,None,0.3,65.40,NaN,NaN,NaN
76514,2,2021-01-31 22:43:00,2021-01-31 23:21:00,None,NaN,35,213,NaN,18.36,46.66,0.00,0.0,12.20,6.12,None,0.3,65.28,NaN,NaN,NaN
76515,2,2021-01-31 22:16:00,2021-01-31 22:27:00,None,NaN,74,69,NaN,2.50,18.95,2.75,0.0,0.00,0.00,None,0.3,22.00,NaN,NaN,NaN
76516,2,2021-01-31 23:10:00,2021-01-31 23:37:00,None,NaN,168,215,NaN,14.48,48.87,2.75,0.0,0.00,6.12,None,0.3,58.04,NaN,NaN,NaN


In [8]:
df_val = nyc.download_data(year=2021, month=2, taxi='green')
df_val

Downloaded 64572 records


,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2021-02-01 00:34:03,2021-02-01 00:51:58,N,1.0,130,205,5.0,3.66,14.00,0.50,0.5,10.00,0.0,None,0.3,25.30,1.0,1.0,0.00
1,2,2021-02-01 00:04:00,2021-02-01 00:10:30,N,1.0,152,244,1.0,1.10,6.50,0.50,0.5,0.00,0.0,None,0.3,7.80,2.0,1.0,0.00
2,2,2021-02-01 00:18:51,2021-02-01 00:34:06,N,1.0,152,48,1.0,4.93,16.50,0.50,0.5,0.00,0.0,None,0.3,20.55,2.0,1.0,2.75
3,2,2021-02-01 00:53:27,2021-02-01 01:11:41,N,1.0,152,241,1.0,6.70,21.00,0.50,0.5,0.00,0.0,None,0.3,22.30,2.0,1.0,0.00
4,2,2021-02-01 00:57:46,2021-02-01 01:06:44,N,1.0,75,42,1.0,1.89,8.50,0.50,0.5,2.45,0.0,None,0.3,12.25,1.0,1.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64567,2,2021-02-28 22:19:00,2021-02-28 22:29:00,None,NaN,129,7,NaN,2.63,10.04,0.00,0.0,0.00,0.0,None,0.3,10.34,NaN,NaN,NaN
64568,2,2021-02-28 23:18:00,2021-02-28 23:27:00,None,NaN,116,166,NaN,1.87,8.33,0.00,0.0,1.89,0.0,None,0.3,10.52,NaN,NaN,NaN
64569,2,2021-02-28 23:44:00,2021-02-28 23:58:00,None,NaN,74,151,NaN,2.40,12.61,0.00,0.0,0.00,0.0,None,0.3,12.91,NaN,NaN,NaN
64570,2,2021-02-28 23:07:00,2021-02-28 23:14:00,None,NaN,42,42,NaN,1.11,11.95,2.75,0.0,0.00,0.0,None,0.3,15.00,NaN,NaN,NaN


In [9]:
len(df_train), len(df_val)

(76518, 64572)

In [10]:
df_train, df_trainy = nyc.prepare_data(df_train)
df_train['duration'] = df_trainy

df_val, df_valy = nyc.prepare_data(df_val)
df_val['duration'] = df_valy

In [11]:
len(df_train), len(df_val)

(73908, 61921)

In [12]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [13]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [14]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [15]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.758715205933011

In [22]:
with open('/home/habeeb/Mlops-proj/02-experimental_tracking/models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [23]:
with mlflow.start_run():

    mlflow.set_tag("developer", "Habeeb")

    mlflow.log_param("train-data", "2021-01-green")
    mlflow.log_param("valid-data", "2021-01-green")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [24]:
import xgboost as xgb

In [26]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [27]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [28]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [29]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

2025-05-12 22:34:20,306 - INFO - build_posterior_wrapper took 0.001372 seconds
2025-05-12 22:34:20,307 - INFO - TPE using 0 trials


[0]	validation-rmse:10.35734                          
[1]	validation-rmse:9.07252                           
[2]	validation-rmse:8.20497                           
[3]	validation-rmse:7.63050                           
[4]	validation-rmse:7.25294                           
[5]	validation-rmse:7.01014                           
[6]	validation-rmse:6.85119                           
[7]	validation-rmse:6.74646                           
[8]	validation-rmse:6.67307                           
[9]	validation-rmse:6.62184                           
[10]	validation-rmse:6.58624                          
[11]	validation-rmse:6.55855                          
[12]	validation-rmse:6.53714                          
[13]	validation-rmse:6.52398                          
[14]	validation-rmse:6.51348                          
[15]	validation-rmse:6.50309                          
[16]	validation-rmse:6.49643                          
[17]	validation-rmse:6.48805                          
[18]	valid

2025-05-12 22:34:29,807 - INFO - build_posterior_wrapper took 0.001283 seconds
2025-05-12 22:34:29,807 - INFO - TPE using 1/1 trials with best loss 6.386670


[0]	validation-rmse:11.44852                                                   
[1]	validation-rmse:10.77741                                                   
[2]	validation-rmse:10.18919                                                   
[3]	validation-rmse:9.67672                                                    
[4]	validation-rmse:9.23073                                                    
[5]	validation-rmse:8.84306                                                    
[6]	validation-rmse:8.50872                                                    
[7]	validation-rmse:8.22215                                                    
[8]	validation-rmse:7.97560                                                    
[9]	validation-rmse:7.76404                                                    
[10]	validation-rmse:7.58481                                                   
[11]	validation-rmse:7.42917                                                   
[12]	validation-rmse:7.29838            

...

[0]	validation-rmse:6.78138                                                     
[1]	validation-rmse:6.61984                                                     
[2]	validation-rmse:6.59980                                                     
[3]	validation-rmse:6.58881                                                     
[4]	validation-rmse:6.57816                                                     
[5]	validation-rmse:6.57117                                                     
[6]	validation-rmse:6.56285                                                     
[7]	validation-rmse:6.55581                                                     
[8]	validation-rmse:6.54850                                                     
[9]	validation-rmse:6.54185                                                     
[10]	validation-rmse:6.53386                                                    
[11]	validation-rmse:6.52902                                                    
[12]	validation-rmse:6.52454

2025-05-12 22:45:38,545 - INFO - build_posterior_wrapper took 0.001467 seconds
2025-05-12 22:45:38,546 - INFO - TPE using 48/48 trials with best loss 6.305974


[0]	validation-rmse:9.61214                                                     
[1]	validation-rmse:8.20614                                                     
[2]	validation-rmse:7.48863                                                     
[3]	validation-rmse:7.13205                                                     
[4]	validation-rmse:6.95136                                                     
[5]	validation-rmse:6.85941                                                     
[6]	validation-rmse:6.80693                                                     
[7]	validation-rmse:6.77413                                                     
[8]	validation-rmse:6.75242                                                     
[9]	validation-rmse:6.73937                                                     
[10]	validation-rmse:6.73093                                                    
[11]	validation-rmse:6.72273                                                    
[12]	validation-rmse:6.71836

2025-05-12 22:45:48,100 - INFO - build_posterior_wrapper took 0.001254 seconds
2025-05-12 22:45:48,102 - INFO - TPE using 49/49 trials with best loss 6.305974


[0]	validation-rmse:10.71345                                                    
[1]	validation-rmse:9.59854                                                     
[2]	validation-rmse:8.78284                                                     
[3]	validation-rmse:8.19641                                                     
[4]	validation-rmse:7.77994                                                     
[5]	validation-rmse:7.48570                                                     
[6]	validation-rmse:7.27950                                                     
[7]	validation-rmse:7.13394                                                     
[8]	validation-rmse:7.03117                                                     
[9]	validation-rmse:6.95649                                                     
[10]	validation-rmse:6.90260                                                    
[11]	validation-rmse:6.86546                                                    
[12]	validation-rmse:6.83736

In [30]:
mlflow.xgboost.autolog(disable=True)

In [31]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("/home/habeeb/Mlops-proj/02-experimental_tracking/models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("/home/habeeb/Mlops-proj/02-experimental_tracking/models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[0]	validation-rmse:11.44482
[1]	validation-rmse:10.77202
[2]	validation-rmse:10.18363
[3]	validation-rmse:9.67396
[4]	validation-rmse:9.23166
[5]	validation-rmse:8.84808
[6]	validation-rmse:8.51883
[7]	validation-rmse:8.23597
[8]	validation-rmse:7.99320
[9]	validation-rmse:7.78709
[10]	validation-rmse:7.61022
[11]	validation-rmse:7.45952
[12]	validation-rmse:7.33049
[13]	validation-rmse:7.22098
[14]	validation-rmse:7.12713
[15]	validation-rmse:7.04752
[16]	validation-rmse:6.98005
[17]	validation-rmse:6.92232
[18]	validation-rmse:6.87112
[19]	validation-rmse:6.82740
[20]	validation-rmse:6.78995
[21]	validation-rmse:6.75792
[22]	validation-rmse:6.72994
[23]	validation-rmse:6.70547
[24]	validation-rmse:6.68390
[25]	validation-rmse:6.66421
[26]	validation-rmse:6.64806
[27]	validation-rmse:6.63280
[28]	validation-rmse:6.61924
[29]	validation-rmse:6.60773
[30]	validation-rmse:6.59777
[31]	validation-rmse:6.58875
[32]	validation-rmse:6.58107
[33]	validation-rmse:6.57217
[34]	validation-rmse:

2025/05/12 23:16:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [32]:
logged_model = 'runs:/e2a9af7333c045f69e187b02818a21ed/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: e2a9af7333c045f69e187b02818a21ed

In [34]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

In [35]:
xgboost_model.best_score

6.318445790064281

In [36]:
xgboost_model.predict(valid)

array([14.782765 ,  7.184751 , 15.971323 , ..., 13.464008 ,  6.7352147,
        8.184539 ], dtype=float32)

In [16]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        #mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        #mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_param("train-data", "2021-01-green")
        mlflow.log_param("valid-data", "2021-01-green")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

In [33]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [20]:
experiments = client.search_experiments()
print(experiments)
for exp in experiments:
    print(exp.name, exp.experiment_id)

[<Experiment: artifact_location='/home/habeeb/Mlops-proj/02-experimental_tracking/mlruns/1', creation_time=1747104414113, experiment_id='1', last_update_time=1747104414113, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>, <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1747104147970, experiment_id='0', last_update_time=1747104147970, lifecycle_stage='active', name='Default', tags={}>]
nyc-taxi-experiment 1
Default 0


In [22]:
client.create_experiment(name="my-cool-experiment")


'2'

In [47]:
from mlflow.entities import ViewType
runs = client.search_runs(
    experiment_ids=["1"],
    filter_string="metrics.rmse < 9",
    order_by=["metrics.rmse ASC"],
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5
)
print(runs)

[<Run: data=<RunData: metrics={'rmse': 6.305973575716166}, params={'learning_rate': '0.2858837944487236',
 'max_depth': '16',
 'min_child_weight': '1.3592331431169886',
 'objective': 'reg:linear',
 'reg_alpha': '0.011544050502747242',
 'reg_lambda': '0.004210727001486417',
 'seed': '42'}, tags={'mlflow.runName': 'sneaky-shrike-130',
 'mlflow.source.name': '/home/habeeb/miniconda3/envs/serverlessML/lib/python3.12/site-packages/ipykernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'habeeb',
 'model': 'xgboost'}>, info=<RunInfo: artifact_uri='/home/habeeb/Mlops-proj/02-experimental_tracking/mlruns/1/feee00192d8542edb90fc26f900f808a/artifacts', end_time=1747111510885, experiment_id='1', lifecycle_stage='active', run_id='feee00192d8542edb90fc26f900f808a', run_name='sneaky-shrike-130', run_uuid='feee00192d8542edb90fc26f900f808a', start_time=1747111499565, status='FINISHED', user_id='habeeb'>, inputs=<RunInputs: dataset_inputs=[]>>, <Run: data=<RunData: metrics={'rmse': 6.30

In [48]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: feee00192d8542edb90fc26f900f808a, rmse: 6.3060
run id: 6dc2c075143f4d73888829b607d8fea6, rmse: 6.3075
run id: 72e441e82d2844879c77bef1d02e190a, rmse: 6.3085
run id: 6eb91ee1131244c381edfb0582f5605a, rmse: 6.3085
run id: 1fcb5291b46d4aec826bf4031c82b1e6, rmse: 6.3090


In [28]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [29]:
# import mlflow
# import mlflow.sklearn
# from sklearn.ensemble import RandomForestRegressor

# with mlflow.start_run() as run:
#     model = RandomForestRegressor().fit([[0, 0]], [0])
#     mlflow.sklearn.log_model(
#         sk_model=model,
#         artifact_path="sklearn-model",
#         registered_model_name="my-registered-model"
#     )

In [ ]:
run_id = "6dc2c075143f4d73888829b607d8fea6"
model_uri = f"runs:/{run_id}/model"
result = mlflow.register_model(model_uri=model_uri, name="nye-taxi-regressor")

In [49]:
model_name = "nye-taxi-regressor"
reg_models = client.get_registered_model(name=model_name)
for version in reg_models.latest_versions:
    print(version)

<ModelVersion: aliases=[], creation_timestamp=1747190361930, current_stage='None', description='', last_updated_timestamp=1747190361930, name='nye-taxi-regressor', run_id='8948732489eb401b8d76ef95404af0a4', run_link='', source='/home/habeeb/Mlops-proj/02-experimental_tracking/mlruns/1/8948732489eb401b8d76ef95404af0a4/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=5>
<ModelVersion: aliases=[], creation_timestamp=1747190306618, current_stage='Production', description='', last_updated_timestamp=1747199929178, name='nye-taxi-regressor', run_id='ff6829ed9d984ebcb2bb77d7bb3d43d0', run_link='', source='/home/habeeb/Mlops-proj/02-experimental_tracking/mlruns/1/ff6829ed9d984ebcb2bb77d7bb3d43d0/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>


In [50]:
# latest_versions = client.get_latest_versions(name=model_name)
# for version in latest_versions:
#     print(f"version: {version.version}, stage: {version.current_stage}")

for mv in client.search_model_versions():
    print(mv.name, mv.version, mv.current_stage)

nye-taxi-regressor 2 Production
nye-taxi-regressor 5 None
nye-taxi-regressor 4 None
nye-taxi-regressor 3 None
nye-taxi-regressor 1 None


In [44]:
client.transition_model_version_stage(
    name=model_name,
    version=2,
    stage="Production",
    archive_existing_versions=False  # archives previous versions in this stage
)

<ModelVersion: aliases=[], creation_timestamp=1747190306618, current_stage='Production', description='', last_updated_timestamp=1747199929178, name='nye-taxi-regressor', run_id='ff6829ed9d984ebcb2bb77d7bb3d43d0', run_link='', source='/home/habeeb/Mlops-proj/02-experimental_tracking/mlruns/1/ff6829ed9d984ebcb2bb77d7bb3d43d0/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [45]:
client.set_registered_model_alias(model_name, "production", 2)  # assigns alias "production" to version 2


In [46]:
import mlflow.pyfunc
model = mlflow.pyfunc.load_model(f"models:/{model_name}@production")
model

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.sklearn
  run_id: ff6829ed9d984ebcb2bb77d7bb3d43d0

In [51]:
from datetime import datetime
client.update_model_version(
    name=model_name,
    version=2,
    description=f"Updated description for this model version on {datetime.today().date()}"
)

<ModelVersion: aliases=['production'], creation_timestamp=1747190306618, current_stage='Production', description='Updated description for this model version on 2025-05-13', last_updated_timestamp=1747202088146, name='nye-taxi-regressor', run_id='ff6829ed9d984ebcb2bb77d7bb3d43d0', run_link='', source='/home/habeeb/Mlops-proj/02-experimental_tracking/mlruns/1/ff6829ed9d984ebcb2bb77d7bb3d43d0/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [ ]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [ ]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

In [ ]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [ ]:
X_test, y_test = df_train, df_trainy

In [ ]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

In [ ]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)